In [1]:
from matplotlib import pyplot as plt
import time
import datetime
from utils import *
from data_processing import *
today = datetime.date.today()
input_url = "input/dataset.csv"

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, periods=5, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in//periods, 0, -1):
		cols.append(df.shift(i*periods)-df)
		names += [('var%d(t-%d)' % (j+1, i*periods)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out//periods):
		cols.append(df.shift(-i*periods)-df)
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i*periods)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
def read_cmapss(input_url):
    df = pd.read_csv("input/dataset.csv", header=None)
    columns_old = ["ENGINEID", "TIMECYCLE", "OPSET1", "OPSET2", "OPSET3", "Total temp at fan in (T2)", "Total temp at LPC out (T24)", "Total temp at HPC out (T30)", "Total temp at LPT out (T50)", 
"Pres at fan in (P2)", "Total pres in bypass-duct (P15)", "Total pres at HPC out (P30)", "Physical fan speed (Nf)", 
"Physical core speed (Nc)", "Engine pres ratio (epr=P50/P2)", "Static pres at HPC out (Ps30)", "Ratio of fuel flow to Ps30 (phi)",
"Corrected fan speed (NRf)", "Corrected core speed (NRc)", "Bypass Ratio (BPR)", "Burner fuel-air ratio (farB)", 
"Bleed Enthalpy (htBleed)", "Demanded fan speed (Nf_dmd)", "Demanded corrected fan speed (PCNfR_dmd)", "HPT coolant bleed (W31)",
"LPT coolant bleed (W32)", "FILEID","RUL"]
    columns_new = ["FILEID","ENGINEID", "TIMECYCLE", "OPSET1", "OPSET2", "OPSET3", "Total temp at fan in (T2)", "Total temp at LPC out (T24)", "Total temp at HPC out (T30)", "Total temp at LPT out (T50)", 
"Pres at fan in (P2)", "Total pres in bypass-duct (P15)", "Total pres at HPC out (P30)", "Physical fan speed (Nf)", 
"Physical core speed (Nc)", "Engine pres ratio (epr=P50/P2)", "Static pres at HPC out (Ps30)", "Ratio of fuel flow to Ps30 (phi)",
"Corrected fan speed (NRf)", "Corrected core speed (NRc)", "Bypass Ratio (BPR)", "Burner fuel-air ratio (farB)", 
"Bleed Enthalpy (htBleed)", "Demanded fan speed (Nf_dmd)", "Demanded corrected fan speed (PCNfR_dmd)", "HPT coolant bleed (W31)",
"LPT coolant bleed (W32)", "RUL"]
    df.columns = columns_old
    df = df[columns_new]

    return df
    

In [4]:
df = read_cmapss(input_url)
feat_names = df.columns.values[3:-1]
target_name = df.columns.values[-1]
df[feat_names] = data_norm(df[feat_names])

In [5]:
select_feat = ["Total temp at HPC out (T30)", 
               "Total temp at LPT out (T50)", 
               "Physical core speed (Nc)", 
               "Static pres at HPC out (Ps30)", 
               "Corrected core speed (NRc)", 
               "Bypass Ratio (BPR)", 
               "Bleed Enthalpy (htBleed)"]

In [6]:
train = df[(["FILEID","ENGINEID", "TIMECYCLE"]+select_feat+["RUL"])][df.FILEID.isin([101, 103])]

In [7]:
train.head()

FILEID  ENGINEID  TIMECYCLE  Total temp at HPC out (T30)  \
0     101         1          1                     1.046743   
1     101         1          2                     1.064701   
2     101         1          3                     1.032258   
3     101         1          4                     0.988211   
4     101         1          5                     0.988719   

   Total temp at LPT out (T50)  Physical core speed (Nc)  \
0                     1.036938                  0.991847   
1                     1.055603                  0.986180   
2                     1.063392                  1.009890   
3                     1.046270                  1.000641   
4                     1.078235                  1.015798   

   Static pres at HPC out (Ps30)  Corrected core speed (NRc)  \
0                       0.965896                    0.641976   
1                       0.971745                    0.552846   
2                       0.907411                    0.574597   
3                       0.866472                    0.582098   
4                       0.910336                    0.581723   

   Bypass Ratio (BPR)  Bleed Enthalpy (htBleed)  RUL  
0           -0.842326                  1.017912  120  
1           -0.825933                  1.017912  120  
2           -0.844592                  0.953370  120  
3           -0.910696                  1.017912  120  
4           -0.829132                  1.050183  120

# Model training

## LSTM (Sibur)

In [8]:
timesteps = 30
lag=1

In [27]:
eng_Xy_tuples = train[(train["FILEID"]==101)].groupby("ENGINEID").apply(lambda x:lstm_sampling(x.iloc[:,:-1], x.RUL, timesteps, lag))

Samples length 163
Targets length 163
Samples length 163
Targets length 163
Samples length 258
Targets length 258
Samples length 150
Targets length 150
Samples length 160
Targets length 160
Samples length 240
Targets length 240
Samples length 159
Targets length 159
Samples length 230
Targets length 230
Samples length 121
Targets length 121
Samples length 172
Targets length 172
Samples length 193
Targets length 193
Samples length 211
Targets length 211
Samples length 141
Targets length 141
Samples length 134
Targets length 134
Samples length 151
Targets length 151
Samples length 178
Targets length 178
Samples length 180
Targets length 180
Samples length 247
Targets length 247
Samples length 166
Targets length 166
Samples length 129
Targets length 129
Samples length 205
Targets length 205
Samples length 166
Targets length 166
Samples length 173
Targets length 173
Samples length 139
Targets length 139
Samples length 118
Targets length 118
Samples length 201
Targets length 201
Samples leng

In [36]:
type(eng_Xy_tuples[1])

tuple

In [38]:
train.ENGINEID.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [39]:
    Train = True
    Predict = True
    plot = False

In [ ]:
    batch_size = 1  # Batch size
    shift = 1
    #if Train == False: batch_size = 1

    sequence_length = timesteps  # Number of steps
    learning_rate = 2*10e-5  # 0.0001
    epochs = 1000
    ann_hidden = 16

    n_channels = x_train.shape[2]

    lstm_size = 48  # Number LSTM units
    num_layers = 2  # 2  # Number of layers
    alpha = 0 # regularization coef

In [ ]:
    X = tf.placeholder(tf.float32, [None, sequence_length, n_channels], name='inputs')
    Y = tf.placeholder(tf.float32, [None, sequence_length], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    is_train = tf.placeholder(dtype=tf.bool, shape=None, name="is_train")

    conv_last_layer = X

    shape = conv_last_layer.get_shape().as_list()
    print('My Conv Shape:',shape)
    CNN_flat = tf.reshape(conv_last_layer, [-1, shape[1] * shape[2]])

    dence_layer_1 = dense_layer(CNN_flat, size=sequence_length * n_channels, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_1")
    lstm_input = tf.reshape(dence_layer_1, [-1, sequence_length, n_channels])

    cell = get_RNNCell(['LSTM'] * num_layers, keep_prob=keep_prob, state_size=lstm_size)
    init_states = cell.zero_state(batch_size, tf.float32)
    
    # For each layer, get the initial state. states will be a tuple of LSTMStateTuples.
    states = get_state_variables(batch_size, cell)

    # Unroll the LSTM
    rnn_output, new_states = tf.nn.dynamic_rnn(cell, lstm_input, dtype=tf.float32, initial_state=states)
    
    # Add an operation to update the train states with the last state tensors.
    update_op = get_state_update_op(states, new_states)
    reset_op = get_state_update_op(states, init_states)
    
    stacked_rnn_output = tf.reshape(rnn_output, [-1, lstm_size])  # change the form into a tensor

    dence_layer_2 = dense_layer(stacked_rnn_output, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2")
    
    dence_layer_3 = dense_layer(dence_layer_2, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2_2")

    output = dense_layer(dence_layer_3, size=1, activation_fn=None, batch_norm=False, phase=is_train, drop_out=False,
                         keep_prob=keep_prob,
                         scope="fc_3_output")

    prediction = tf.reshape(output, [-1])
    y_flat = tf.reshape(Y, [-1])

    h = prediction - y_flat
    
    tv = tf.trainable_variables()
    regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])

    cost_function = tf.reduce_sum(tf.square(h)) + alpha*regularization_cost
    RMSE = tf.sqrt(tf.reduce_mean(tf.square(h)))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost_function)

    saver = tf.train.Saver()
    
    training_generator = batch_generator(x_train, y_train, batch_size, sequence_length, online=True, online_shift=shift)
    testing_generator = batch_generator(x_test, y_test, batch_size, sequence_length, online=True, online_shift=shift)
    #print(len(list(training_generator)))

    if Train: model_summary(learning_rate=learning_rate, batch_size=batch_size, lstm_layers=num_layers,
                            lstm_layer_size=lstm_size, fc_layer_size=ann_hidden, sequence_length=sequence_length,
                            n_channels=n_channels, path_checkpoint=path_checkpoint, spacial_note='')

## Gradient Boosting Regression

### TS fresh dataset

In [34]:
check = train[(train["FILEID"]==101) & (train["ENGINEID"]==1)]
X = check[check.columns.values[3:-1]]
y = check["RUL"]

In [35]:
timesteps = 30
lag=1

In [36]:
X, y = lstm_sampling(X, y, timesteps, lag)

Samples length 163
Targets length 163


In [40]:
from tsfresh import select_features, extract_features
from tsfresh.utilities.dataframe_functions import impute

In [16]:
X = df[feat_names]
y = df[target_name]

In [17]:
test = df[df["FILEID"]==101][df["ENGINEID"]==1].copy()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [18]:
test=test.drop(["FILEID", "ENGINEID"], axis=1)

In [19]:
def add_lag_roll(df, feat_names, lag=1, step=1, roll=0):
    x = series_to_supervised(test[feat_names], n_in=lag, n_out=1, periods=step)    
    return pd.concat([df[lag:], x], axis=1)
    

In [20]:
lag = 20
step = 2

In [21]:
add_lag_roll(test.iloc[:,:-2], feat_names, lag=lag, step=step, roll=0).head()

TIMECYCLE    OPSET1    OPSET2   OPSET3  Total temp at fan in (T2)  \
20         21 -1.042166 -1.114839  0.34552                   1.079737   
21         22 -1.042081 -1.115110  0.34552                   1.079737   
22         23 -1.041887 -1.115926  0.34552                   1.079737   
23         24 -1.042153 -1.114295  0.34552                   1.079737   
24         25 -1.041954 -1.116197  0.34552                   1.079737   

    Total temp at LPC out (T24)  Total temp at HPC out (T30)  \
20                     1.061976                     1.015995   
21                     1.071397                     1.074104   
22                     1.056559                     1.033953   
23                     1.062211                     1.056315   
24                     1.071397                     1.084014   

    Total temp at LPT out (T50)  Pres at fan in (P2)  \
20                     1.018788             1.108381   
21                     1.036718             1.108381   
22                     0.993951             1.108381   
23                     1.023785             1.108381   
24                     1.028047             1.108381   

    Total pres in bypass-duct (P15)     ...      var15(t-2)  var16(t-2)  \
20                         1.115795     ...       -0.003229    -0.00045   
21                         1.115795     ...        0.000426    -0.00009   
22                         1.115795     ...        0.005482     0.00027   
23                         1.115795     ...       -0.000792     0.00000   
24                         1.115795     ...       -0.006335     0.00027   

    var17(t-2)  var18(t-2)  var19(t-2)  var20(t-2)  var21(t-2)  var22(t-2)  \
20   -0.061129    0.036251         0.0   -0.032271         0.0         0.0   
21   -0.008751    0.023456         0.0    0.000000         0.0         0.0   
22    0.076505   -0.025455         0.0    0.000000         0.0         0.0   
23   -0.043378    0.019058         0.0    0.000000         0.0         0.0   
24   -0.108882    0.043048         0.0   -0.032271         0.0         0.0   

    var23(t-2)  var24(t-2)  
20   -0.024797    0.005173  
21    0.009406    0.006099  
22    0.012826   -0.020820  
23   -0.006840    0.001368  
24   -0.000855    0.003905  

[5 rows x 264 columns]

In [22]:
#df.groupby(["FILEID", "ENGINEID"])

In [23]:
#X=pd.DataFrame()
#y=list()

In [24]:
#for name, group in df.groupby(["FILEID", "ENGINEID"]):
    #X = pd.concat([X, add_lag_roll(group[feat_names], feat_names, lag=lag, step=step)], axis=0)
    #y = y.append(group["RUL"][lag:])
    
    

In [25]:
X.tail(1).values

array([[ 1.07587484,  1.16808529,  0.34551956, -1.19585285, -0.96277102,
        -0.74158237, -0.87287892, -1.03437167, -0.99635436, -0.94961297,
        -0.35376305, -0.75431935, -0.93920659, -0.61027389, -0.95498265,
         0.35008025,  0.13632058,  0.45963462, -1.02579284, -0.8215301 ,
        -0.35667189,  0.34551956, -0.9598422 , -0.95726673]])